In [20]:
# -*- coding: utf-8 -*-
"""Untitled23.ipynb

Automatically generated by Colab.

Original file is located at
    https://colab.research.google.com/drive/1gyqSXuq-Wga83Gkhpaox72ljcg00I8WQ
"""

import requests
from bs4 import BeautifulSoup
import re
from nltk.stem import PorterStemmer
from urllib.parse import urljoin
import time
from collections import defaultdict
import math  # For log in IDF
import pandas as pd
from IPython.display import HTML, display, clear_output
import ipywidgets as widgets
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

class SearchEngine:
    def __init__(self):
        self.stemmer = PorterStemmer()
        self.stop_words = {
            'a', 'about', 'above', 'after', 'again', 'against', 'all', 'am', 'an', 'and', 'any',
            'are', 'aren', 'as', 'at', 'be', 'because', 'been', 'before', 'being', 'below', 'between',
            'both', 'but', 'by', 'can', 'could', 'did', 'do', 'does', 'doing', 'don', 'down', 'during',
            'each', 'few', 'for', 'from', 'further', 'had', 'has', 'have', 'having', 'he', 'her',
            'here', 'hers', 'herself', 'him', 'himself', 'his', 'how', 'i', 'if', 'in', 'into', 'is',
            'it', 'its', 'itself', 'just', 'me', 'more', 'most', 'my', 'myself', 'no', 'nor', 'not',
            'now', 'of', 'off', 'on', 'once', 'only', 'or', 'other', 'our', 'ours', 'ourselves', 'out',
            'over', 'own', 's', 'same', 'she', 'should', 'so', 'some', 'such', 'than', 'that', 'the',
            'their', 'theirs', 'them', 'themselves', 'then', 'there', 'these', 'they', 'this', 'those',
            'through', 'to', 'too', 'under', 'until', 'up', 'very', 'was', 'we', 'were', 'what', 'when',
            'where', 'which', 'while', 'who', 'whom', 'why', 'will', 'with', 'you', 'your', 'yours',
            'yourself', 'yourselves'
        }
        self.url_to_id = {}
        self.id_to_url = {}
        self.next_id = 1

        # inverted_index[word][doc_id] = count of word in doc
        self.inverted_index = defaultdict(lambda: defaultdict(int))

        # global_index[word][url] = count of word in url
        self.global_index = defaultdict(lambda: defaultdict(int))

    def analyze_link_structure(self, max_iterations=20, epsilon=1e-8):
        """
        Implements the HITS algorithm to find Hubs and Authorities
        max_iterations: Maximum number of iterations for the algorithm
        epsilon: Convergence threshold
        """
        # Initialize dictionaries for the adjacency matrix
        outgoing_links = defaultdict(set)  # pages that page i points to
        incoming_links = defaultdict(set)  # pages that point to page i

        # Build the adjacency matrix
        for url_id in self.id_to_url:
            url = self.id_to_url[url_id]
            soup = self.fetch_page(url)
            if soup:
                links = self.get_links(soup, url)
                for link in links:
                    if link in self.url_to_id:
                        target_id = self.url_to_id[link]
                        outgoing_links[url_id].add(target_id)
                        incoming_links[target_id].add(url_id)

        # Initialize hub and authority scores
        hub_scores = {url_id: 1.0 for url_id in self.id_to_url}
        auth_scores = {url_id: 1.0 for url_id in self.id_to_url}

        # HITS algorithm iterations
        for _ in range(max_iterations):
            # Store old scores for convergence check
            old_hub_scores = hub_scores.copy()
            old_auth_scores = auth_scores.copy()

            # Update authority scores
            auth_scores = {url_id: 0.0 for url_id in self.id_to_url}
            for page in self.id_to_url:
                auth_scores[page] = sum(hub_scores[incoming] for incoming in incoming_links[page])

            # Normalize authority scores
            norm = math.sqrt(sum(score * score for score in auth_scores.values()))
            if norm > 0:
                auth_scores = {page: score/norm for page, score in auth_scores.items()}

            # Update hub scores
            hub_scores = {url_id: 0.0 for url_id in self.id_to_url}
            for page in self.id_to_url:
                hub_scores[page] = sum(auth_scores[outgoing] for outgoing in outgoing_links[page])

            # Normalize hub scores
            norm = math.sqrt(sum(score * score for score in hub_scores.values()))
            if norm > 0:
                hub_scores = {page: score/norm for page, score in hub_scores.items()}

            # Check convergence
            auth_diff = sum(abs(auth_scores[page] - old_auth_scores[page]) for page in self.id_to_url)
            hub_diff = sum(abs(hub_scores[page] - old_hub_scores[page]) for page in self.id_to_url)

            if auth_diff < epsilon and hub_diff < epsilon:
                print(f"Converged after {_+1} iterations")
                break

        # Print analysis results
        print("\n=== HITS Algorithm Analysis ===")
        print("\nTop Hubs (pages that point to good authorities):")
        sorted_hubs = sorted(hub_scores.items(), key=lambda x: x[1], reverse=True)
        for url_id, score in sorted_hubs[:5]:
            print(f"URL ID: {url_id}, URL: {self.id_to_url[url_id]}")
            print(f"Hub Score: {score:.6f}")
            print(f"Number of Outgoing Links: {len(outgoing_links[url_id])}")

        print("\nTop Authorities (pages pointed to by good hubs):")
        sorted_authorities = sorted(auth_scores.items(), key=lambda x: x[1], reverse=True)
        for url_id, score in sorted_authorities[:5]:
            print(f"URL ID: {url_id}, URL: {self.id_to_url[url_id]}")
            print(f"Authority Score: {score:.6f}")
            print(f"Number of Incoming Links: {len(incoming_links[url_id])}")

        # Return both scores and link information
        return {
            'hub_scores': hub_scores,
            'auth_scores': auth_scores,
            'outgoing_links': outgoing_links,
            'incoming_links': incoming_links
        }

    def get_url_id(self, url):
        """Map URLs to unique IDs"""
        if url not in self.url_to_id:
            self.url_to_id[url] = self.next_id
            self.id_to_url[self.next_id] = url
            self.next_id += 1
        return self.url_to_id[url]

    def fetch_page(self, url):
        """Fetch the HTML page content"""
        try:
            time.sleep(1)
            response = requests.get(url, timeout=10)
            if response.status_code == 200:
                return BeautifulSoup(response.text, 'html.parser')
            else:
                print(f"Failed to fetch {url}: Status code {response.status_code}")
                return None
        except Exception as e:
            print(f"Error fetching {url}: {str(e)}")
            return None

    def get_links(self, soup, base_url):
        """Extract links from a page"""
        links = set()
        if soup:
            for link in soup.find_all('a', href=True):
                url = link['href']
                absolute_url = urljoin(base_url, url)
                if absolute_url.startswith(base_url):
                    links.add(absolute_url)
        return links

    def index_words(self, soup, url):
        """
        Index words from a page, storing the stemmed form.
        Returns a dictionary of {word: count_in_this_document}.
        """
        index = defaultdict(int)
        url_id = self.get_url_id(url)

        if soup:
            words = re.findall(r'\w+', soup.get_text())
            for word in words:
                word = word.lower()
                if word not in self.stop_words:
                    stemmed_word = self.stemmer.stem(word)

                    # Increase counters in the local index (for this page)
                    index[stemmed_word] += 1

                    # Update the global/inverted index
                    self.inverted_index[stemmed_word][url_id] += 1
                    self.global_index[stemmed_word][url] += 1

        return dict(index)

    def crawl_and_index(self, start_url, max_pages=5):
        """Crawl and index pages. Returns a dict: {doc_id: {word: count}}"""
        visited = set()
        to_visit = {start_url}
        page_indexes = {}

        while to_visit and len(visited) < max_pages:
            url = to_visit.pop()
            if url in visited:
                continue

            print(f"Crawling: {url} (ID: {self.get_url_id(url)})")
            soup = self.fetch_page(url)
            if soup:
                page_indexes[self.get_url_id(url)] = self.index_words(soup, url)
                visited.add(url)
                links = self.get_links(soup, start_url)
                to_visit.update(links - visited)

        return page_indexes

    def save_to_excel(self, page_indexes, filename="results.xlsx"):
        """
        Save:
         1) URL Mappings
         2) Page Word Counts
         3) Top 15 Inverted Index
        """
        # 1) URL Mappings sheet
        url_mappings = [{"URL ID": url_id, "URL": url} for url, url_id in self.url_to_id.items()]
        url_df = pd.DataFrame(url_mappings)

        # 2) Page Word Counts sheet
        rows = []
        for url_id, index in page_indexes.items():
            for word, count in index.items():
                rows.append({"URL ID": url_id, "Word": word, "Count": count})
        counts_df = pd.DataFrame(rows)

        # 3) Calculate global word frequencies to find top 15
        word_frequencies = defaultdict(int)
        for word, docs in self.inverted_index.items():
            word_frequencies[word] = sum(docs.values())

        top_words = sorted(word_frequencies.items(), key=lambda x: x[1], reverse=True)[:15]
        top_words_set = {word for word, _ in top_words}

        # Filter the inverted index for the top 15 words
        inverted_rows = []
        for word, docs in self.inverted_index.items():
            if word in top_words_set:
                document_ids = ", ".join(map(str, docs.keys()))
                inverted_rows.append({"Word": word, "Document IDs": document_ids})
        inverted_df = pd.DataFrame(inverted_rows)

        # Save to Excel (initial sheets)
        with pd.ExcelWriter(filename, engine="openpyxl") as writer:
            url_df.to_excel(writer, index=False, sheet_name="URL Mappings")
            counts_df.to_excel(writer, index=False, sheet_name="Page Word Counts")
            inverted_df.to_excel(writer, index=False, sheet_name="Top 15 Inverted Index")

        print(f"Results saved to '{filename}'.")

    def calculate_tf_idf_for_query(self, query, page_indexes, filename="results.xlsx"):
        """
        1) Preprocess query (tokenize, remove stopwords, stemming).
        2) For each query word (concept), for each document:
             - Compute TF
             - Compute IDF
             - Compute TF-IDF
        3) Save detailed results (doc_id, url, concept, TF, IDF, TF-IDF) to a new sheet in the Excel file
           with 10 decimal places.
        """
        # 1) Preprocess the query
        query_words = re.findall(r'\w+', query.lower())
        processed_query = []
        for qw in query_words:
            if qw not in self.stop_words:
                stemmed_qw = self.stemmer.stem(qw)
                processed_query.append(stemmed_qw)

        # 2) Collect stats for TF-IDF
        N = len(page_indexes)  # total number of documents
        doc_total_words = {}
        for doc_id, index_dict in page_indexes.items():
            total_count = sum(index_dict.values())
            doc_total_words[doc_id] = total_count

        # 3) Compute TF, IDF, TF-IDF
        tf_idf_rows = []
        for doc_id, index_dict in page_indexes.items():
            url = self.id_to_url[doc_id]
            total_words_in_doc = doc_total_words[doc_id]

            for qw in processed_query:
                freq_in_doc = self.inverted_index[qw].get(doc_id, 0)
                tf = freq_in_doc / total_words_in_doc if total_words_in_doc > 0 else 0.0

                df = len(self.inverted_index[qw].keys())  # number of docs containing qw
                idf = math.log(N / df, 10) if df > 0 else 0.0
                tf_idf = tf * idf

                tf_idf_rows.append({
                    "Document ID": doc_id,
                    "URL": url,
                    "Query Concept": qw,
                    "TF": tf,
                    "IDF": idf,
                    "TF-IDF": tf_idf
                })

        # 4) Create a DataFrame and append to the existing Excel file
        tf_idf_df = pd.DataFrame(tf_idf_rows)

        try:
            with pd.ExcelWriter(filename, engine="openpyxl", mode="a", if_sheet_exists="replace") as writer:
                tf_idf_df.to_excel(writer, index=False, sheet_name="Query TF-IDF")
            print(f"TF-IDF results for the query saved to '{filename}'.")
        except FileNotFoundError:
            with pd.ExcelWriter(filename, engine="openpyxl") as writer:
                tf_idf_df.to_excel(writer, index=False, sheet_name="Query TF-IDF")
            print(f"File '{filename}' was created and TF-IDF results were saved.")

def select_connected_pages(page_indexes, url_to_id, id_to_url, soup_cache=None, max_pages=10):
    """
    Select up to 10 pages that have links between them from the crawled pages
    Returns: Dictionary of page IDs and their outgoing links to other selected pages
    """
    connected_pages = {}
    page_links = {}

    # Build initial link structure
    for url_id in id_to_url:
        url = id_to_url[url_id]
        soup = soup_cache.get(url) if soup_cache else None
        if soup:
            links = set()
            for link in soup.find_all('a', href=True):
                absolute_url = urljoin(url, link['href'])
                if absolute_url in url_to_id:
                    target_id = url_to_id[absolute_url]
                    links.add(target_id)
            if links:  # Only include pages that have outgoing links
                page_links[url_id] = list(links)

    # Select connected pages (prefer pages with more connections)
    selected_ids = set()
    for page_id, links in sorted(page_links.items(), key=lambda x: len(x[1]), reverse=True):
        if len(selected_ids) >= max_pages:
            break
        connected_links = [l for l in links if l in page_links]
        if connected_links:
            selected_ids.add(page_id)
            selected_ids.update(connected_links[:max_pages-len(selected_ids)])

    # Create final link structure for selected pages
    for page_id in selected_ids:
        outgoing = [l for l in page_links.get(page_id, []) if l in selected_ids]
        if outgoing:
            connected_pages[page_id] = outgoing

    return connected_pages

def calculate_new_pagerank(current_ranks, links):
    """Calculate new PageRank values for a single iteration"""
    new_ranks = {}
    # Calculate new PageRank for each page
    for page in current_ranks:
        # Find who links to this page
        incoming_links = [p for p, outgoing in links.items() if page in outgoing]
        # Sum up PageRank contributions
        rank_sum = 0
        for source_page in incoming_links:
            # Get number of outgoing links from source page
            num_outgoing = len(links[source_page])
            # Add contribution from this source page
            if num_outgoing > 0:  # Avoid division by zero
                rank_sum += current_ranks[source_page] / num_outgoing
        new_ranks[page] = rank_sum
    return new_ranks

def print_ranks(ranks, iteration, id_to_url=None):
    """Print PageRank values after each iteration"""
    print(f"\nPageRank values after iteration {iteration}:")
    print("-" * 50)
    if id_to_url:
        print("Page ID | URL | PageRank Value")
    else:
        print("Page | PageRank Value")
    print("-" * 50)
    for page, rank in ranks.items():
        if id_to_url:
            url = id_to_url[page]
            print(f"{page} | {url[:50]}... | {rank:.3f}")
        else:
            print(f"{page} | {rank:.3f}")
    print("-" * 50)

def run_pagerank_analysis(page_indexes, url_to_id, id_to_url, soup_cache, num_iterations=2):
    """Run complete PageRank analysis on the crawled pages"""
    print("\n=== PageRank Analysis ===")

    # Select connected pages
    connected_pages = select_connected_pages(page_indexes, url_to_id, id_to_url, soup_cache)

    if not connected_pages:
        print("No connected pages found. PageRank analysis cannot proceed.")
        return

    # Show selected pages
    print("\nSelected pages for PageRank analysis:")
    for page_id in connected_pages:
        print(f"\nPage ID: {page_id}, URL: {id_to_url[page_id]}")
        print(f"Links to: {[id_to_url[link_id] for link_id in connected_pages[page_id]]}")

    # Initialize PageRank values
    current_ranks = {page: 1/len(connected_pages) for page in connected_pages}

    # Print initial values
    print("\nInitial PageRank Values:")
    print_ranks(current_ranks, 0, id_to_url)

    # Perform iterations
    for i in range(num_iterations):
        current_ranks = calculate_new_pagerank(current_ranks, connected_pages)
        print_ranks(current_ranks, i + 1, id_to_url)

    # Find highest PageRank
    highest_page = max(current_ranks.items(), key=lambda x: x[1])
    print(f"\nHighest PageRank after {num_iterations} iterations:")
    print(f"Page ID: {highest_page[0]}")
    print(f"URL: {id_to_url[highest_page[0]]}")
    print(f"PageRank value: {highest_page[1]:.3f}")

    return current_ranks

class FeedbackRelevance:
    def __init__(self, pages, pagerank_scores, id_to_url, search_engine):
        self.pages = pages  # List of page IDs
        self.pagerank_scores = pagerank_scores
        self.id_to_url = id_to_url
        self.search_engine = search_engine  # Reference to SearchEngine instance
        self.user_feedback = {}  # Dictionary to store feedback per user
        self.outputs = {}  # Dictionary to store outputs per user
        self.button_pairs = {}  # Dictionary to store button pairs

    def initialize_user(self, user_id):
        """Initialize feedback and output widgets for a new user"""
        if user_id not in self.user_feedback:
            self.user_feedback[user_id] = {}
            self.outputs[user_id] = widgets.Output()
            self.button_pairs[user_id] = {}

    def create_page_display(self, page_id):
        """Create HTML display for a page"""
        url = self.id_to_url[page_id]
        # Get the page content from the search engine's index
        words = self.search_engine.global_index.get(url, {}).keys()
        page_text = " ".join(words)
        return f"""
        <div style="margin: 20px; padding: 10px; border: 1px solid #ddd; border-radius: 5px;">
            <div style="float: right; color: #666;">PageRank Score: {self.pagerank_scores[page_id]:.3f}</div>
            <div style="color: #1a0dab; font-size: 18px;">Page ID: {page_id}</div>
            <div style="color: #006621; font-size: 14px;">{url}</div>
            <div style="color: #545454; margin-top: 5px;">
                Content Preview: {page_text[:200]}...
            </div>
        </div>
        """

    def on_button_click(self, b):
        """Generic handler for button clicks with visual feedback"""
        user_id = int(b.user_id)
        page_id = int(b.page_id)
        is_relevant = b.description.startswith('👍')

        # Update feedback
        self.user_feedback[user_id][page_id] = is_relevant

        # Update button styles
        rel_btn, nonrel_btn = self.button_pairs[user_id][page_id]
        if is_relevant:
            rel_btn.button_style = 'success'
            nonrel_btn.button_style = ''
        else:
            rel_btn.button_style = ''
            nonrel_btn.button_style = 'danger'

        # Show current feedback status
        with self.outputs[user_id]:
            clear_output()
            print(f"✓ Marked document {page_id} as {'relevant' if is_relevant else 'not relevant'}")
            if self.user_feedback[user_id]:
                print("\nCurrent feedback:")
                for pid, v in sorted(self.user_feedback[user_id].items()):
                    status = "relevant" if v else "not relevant"
                    print(f"Document {pid}: {status}")

    def rerank_results(self, user_id):
        """Rerank results based on user feedback using vector space model"""
        relevant_pages = [i for i, v in self.user_feedback[user_id].items() if v == True]
        non_relevant_pages = [i for i, v in self.user_feedback[user_id].items() if v == False]

        # Create TF-IDF vectors
        all_words = set()
        page_word_counts = {}

        # Calculate document frequencies (DF)
        doc_freq = defaultdict(int)
        for page_id in self.pages:
            word_counts = {word: counts[page_id] for word, counts in self.search_engine.inverted_index.items()
                        if page_id in counts}
            for word in word_counts:
                doc_freq[word] += 1
            all_words.update(word_counts.keys())
            page_word_counts[page_id] = word_counts

        # Calculate total words per document
        doc_total_words = {}
        for doc_id, word_counts in page_word_counts.items():
            doc_total_words[doc_id] = sum(word_counts.values())

        # Create word-to-index mapping
        word_to_idx = {word: idx for idx, word in enumerate(sorted(all_words))}

        # Create TF-IDF vectors
        doc_vectors = []
        N = len(self.pages)  # Total number of documents

        for page_id in self.pages:
            vector = np.zeros(len(all_words))
            word_counts = page_word_counts[page_id]
            total_words = doc_total_words[page_id]

            for word, count in word_counts.items():
                idx = word_to_idx[word]
                tf = count / total_words if total_words > 0 else 0
                idf = math.log(N / doc_freq[word]) if doc_freq[word] > 0 else 0
                vector[idx] = tf * idf

            doc_vectors.append(vector)

        doc_vectors = np.array(doc_vectors)

        # Initialize query vector as mean of all documents
        query_vector = np.mean(doc_vectors, axis=0)

        # Modify query vector based on feedback
        beta = 0.75  # Weight for relevant documents
        gamma = 0.15  # Weight for non-relevant documents

        if relevant_pages:
            relevant_indices = [self.pages.index(page_id) for page_id in relevant_pages]
            query_vector += beta * np.mean(doc_vectors[relevant_indices], axis=0)

        if non_relevant_pages:
            non_relevant_indices = [self.pages.index(page_id) for page_id in non_relevant_pages]
            query_vector -= gamma * np.mean(doc_vectors[non_relevant_indices], axis=0)

        # Calculate similarities using sklearn's cosine_similarity
        similarities = cosine_similarity(doc_vectors, query_vector.reshape(1, -1)).flatten()
        ranked_indices = np.argsort(-similarities)

        return [self.pages[i] for i in ranked_indices], similarities[ranked_indices]
    def show_ranking(self, b):
        """Display reranked results with detailed feedback"""
        user_id = int(b.user_id)

        if not self.user_feedback[user_id]:
            with self.outputs[user_id]:
                clear_output()
                print("Please mark at least one document as relevant or not relevant.")
                return

        reranked_pages, scores = self.rerank_results(user_id)

        with self.outputs[user_id]:
            clear_output()
            print(f"🔄 Reranked results for User {user_id}:\n")
            for page_id, score in zip(reranked_pages, scores):
                display(HTML(f"""
                    <div style="margin: 20px; padding: 10px; border: 1px solid #ddd; border-radius: 5px;">
                        <div style="float: right; color: #666;">
                            New Score: {score:.3f} | Original PageRank: {self.pagerank_scores[page_id]:.3f}
                        </div>
                        <div style="color: #1a0dab; font-size: 18px;">Page ID: {page_id}</div>
                        <div style="color: #006621; font-size: 14px;">{self.id_to_url[page_id]}</div>
                        <div style="color: #545454; margin-top: 5px;">
                            Feedback: {
                                "Relevant ✓" if self.user_feedback[user_id].get(page_id) == True
                                else "Not Relevant ✗" if self.user_feedback[user_id].get(page_id) == False
                                else "No Feedback"
                            }
                        </div>
                    </div>
                """))

    def display_interface(self, user_id):
        """Display the feedback interface for a user"""
        self.initialize_user(user_id)

        # Create header
        display(HTML(f"""
            <div style="margin: 20px 0; padding: 10px; background-color: #f8f9fa; border-radius: 5px;">
                <h3 style="margin: 0; color: #333;">Feedback Interface for User {user_id}</h3>
                <p style="margin: 10px 0 0 0; color: #666;">
                    Click 👍 or 👎 to mark pages as relevant or not relevant, then use the buttons below
                    to see reordered results or get query suggestions.
                </p>
            </div>
        """))

        for page_id in self.pages:
            display(HTML(self.create_page_display(page_id)))

            # Create buttons
            rel_btn = widgets.Button(
                description='👍 Relevant',
                tooltip=str(page_id),
                layout=widgets.Layout(margin='5px')
            )
            nonrel_btn = widgets.Button(
                description='👎 Not Relevant',
                tooltip=str(page_id),
                layout=widgets.Layout(margin='5px')
            )

            # Add user_id and page_id as attributes
            rel_btn.user_id = str(user_id)
            rel_btn.page_id = str(page_id)
            nonrel_btn.user_id = str(user_id)
            nonrel_btn.page_id = str(page_id)

            # Store button pair
            self.button_pairs[user_id][page_id] = (rel_btn, nonrel_btn)

            # Set up click handlers
            rel_btn.on_click(self.on_button_click)
            nonrel_btn.on_click(self.on_button_click)

            display(widgets.HBox([rel_btn, nonrel_btn]))

        # Create action buttons container
        button_container = widgets.HBox([
            widgets.Button(
                description='Show Ranking',
                button_style='primary',
                layout=widgets.Layout(margin='20px 10px 20px 0')
            ),
            widgets.Button(
                description='Show Query Suggestion',
                button_style='info',
                layout=widgets.Layout(margin='20px 0 20px 10px')
            )
        ])

        # Set up button attributes and handlers
        rank_button = button_container.children[0]
        suggest_button = button_container.children[1]

        rank_button.user_id = str(user_id)
        suggest_button.user_id = str(user_id)

        rank_button.on_click(self.show_ranking)
        suggest_button.on_click(self.display_query_suggestion)

        display(button_container)
        display(self.outputs[user_id])

    def suggest_expanded_query(self, user_id):
        """
        Generate an expanded query based on user feedback and term weights
        """
        # Get relevant and non-relevant documents
        relevant_pages = [i for i, v in self.user_feedback[user_id].items() if v == True]
        non_relevant_pages = [i for i, v in self.user_feedback[user_id].items() if v == False]

        if not relevant_pages:
            return None, None

        # Collect terms from relevant documents
        term_weights = defaultdict(float)
        term_counts = defaultdict(int)

        # Calculate term frequencies in relevant documents
        for page_id in relevant_pages:
            url = self.id_to_url[page_id]
            # Get terms from the inverted index
            for term, doc_counts in self.search_engine.inverted_index.items():
                if page_id in doc_counts:
                    term_weights[term] += doc_counts[page_id]
                    term_counts[term] += 1

        # Penalize terms that appear in non-relevant documents
        for page_id in non_relevant_pages:
            url = self.id_to_url[page_id]
            for term, doc_counts in self.search_engine.inverted_index.items():
                if page_id in doc_counts:
                    term_weights[term] -= doc_counts[page_id] * 0.5  # Penalty factor

        # Calculate final term weights using Rocchio-like approach
        alpha = 1.0  # Original query weight
        beta = 0.75  # Relevant documents weight
        gamma = 0.15  # Non-relevant documents weight

        # Sort terms by weight and get top N terms
        N = 5  # Number of terms to suggest
        suggested_terms = sorted(
            [(term, weight) for term, weight in term_weights.items()],
            key=lambda x: x[1],
            reverse=True
        )[:N]

        # Create expanded query suggestion
        expansion_terms = []
        term_explanations = []

        for term, weight in suggested_terms:
            if weight > 0:
                expansion_terms.append(term)
                explanation = (
                    f"'{term}' (weight: {weight:.2f}, "
                    f"appears in {term_counts[term]} relevant documents)"
                )
                term_explanations.append(explanation)

        if not expansion_terms:
            return None, None

        expanded_query = " ".join(expansion_terms)
        explanations = "\n".join(f"- {exp}" for exp in term_explanations)

        return expanded_query, explanations

    def display_query_suggestion(self, b):
        """
        Display suggested query based on user feedback
        """
        user_id = int(b.user_id)

        if not self.user_feedback[user_id]:
            with self.outputs[user_id]:
                clear_output()
                print("Please provide feedback on some documents first.")
                return

        expanded_query, explanations = self.suggest_expanded_query(user_id)

        with self.outputs[user_id]:
            clear_output()
            if expanded_query:
                print("📝 Suggested Query Expansion:")
                print(f"\nExpanded Query: {expanded_query}")
                print("\nTerm Selection Explanation:")
                print(explanations)
                print("\nThis suggestion is based on the terms that appear frequently in")
                print("documents you marked as relevant, while avoiding terms from")
                print("non-relevant documents.")
            else:
                print("Unable to generate query suggestion.")
                print("Please mark more documents as relevant/non-relevant.")

    def add_suggestion_button(self):
        """
        Add a button to show query suggestions
        """
        suggest_button = widgets.Button(
            description='Show Query Suggestion',
            button_style='info',
            layout=widgets.Layout(margin='20px 0')
        )
        suggest_button.user_id = str(user_id)
        suggest_button.on_click(self.display_query_suggestion)
        display(suggest_button)

    def implement_feedback_relevance(pagerank_scores, id_to_url, search_engine):
        """Main function to implement feedback relevance"""
        pages = list(pagerank_scores.keys())
        feedback_system = FeedbackRelevance(pages, pagerank_scores, id_to_url, search_engine)

        # Display interface for two users
        print("\nDisplaying Feedback Interface for User 1...")
        feedback_system.display_interface(1)

        print("\nDisplaying Feedback Interface for User 2...")
        feedback_system.display_interface(2)

        return feedback_system

def main():
    search_engine = SearchEngine()
    start_url = "https://www.bundesgesundheitsministerium.de/en/"
    soup_cache = {}  # Cache for storing fetched pages

    print("Starting crawling and indexing...")
    start_crawl = time.time()

    # Modified crawling to store soup objects
    visited = set()
    to_visit = {start_url}
    page_indexes = {}

    while to_visit and len(visited) < 20:  # max_pages=20
        url = to_visit.pop()
        if url in visited:
            continue

        print(f"Crawling: {url} (ID: {search_engine.get_url_id(url)})")
        soup = search_engine.fetch_page(url)
        if soup:
            soup_cache[url] = soup  # Store soup object in cache
            page_indexes[search_engine.get_url_id(url)] = search_engine.index_words(soup, url)
            visited.add(url)
            links = search_engine.get_links(soup, start_url)
            to_visit.update(links - visited)

    end_crawl = time.time()
    crawl_time = end_crawl - start_crawl
    print(f"Crawling and indexing completed in {crawl_time:.2f} seconds.")

    # Run HITS Algorithm Analysis
    link_analysis = search_engine.analyze_link_structure(max_iterations=20, epsilon=1e-8)

    # Run PageRank analysis
    pagerank_scores = run_pagerank_analysis(
        page_indexes,
        search_engine.url_to_id,
        search_engine.id_to_url,
        soup_cache,
        num_iterations=2
    )

    # Continue with existing functionality...
    search_engine.save_to_excel(page_indexes, "initial_results.xlsx")

    queries = [
        "What are the average waiting times in hospitals in Germany by region?",
        "What are the available healthcare policies for chronic diseases in Germany?",
        "How does Germany's healthcare system compare to other EU countries in terms of patient satisfaction?"
    ]

    for i, query in enumerate(queries, start=1):
        print(f"Processing query {i}: {query}")
        start_tfidf = time.time()
        filename = f"results_query_{i}.xlsx"
        search_engine.calculate_tf_idf_for_query(query, page_indexes, filename)
        end_tfidf = time.time()
        tfidf_time = end_tfidf - start_tfidf

        with pd.ExcelWriter(filename, engine="openpyxl", mode="a", if_sheet_exists="replace") as writer:
            timing_data = pd.DataFrame([{
                "Crawling Time (seconds)": crawl_time,
                "TF-IDF Calculation Time (seconds)": tfidf_time
            }])
            timing_data.to_excel(writer, index=False, sheet_name="Timing Information")

        print(f"TF-IDF calculation for query {i} completed in {tfidf_time:.2f} seconds.")
        print(f"Results saved to '{filename}'.")
        print("-" * 80)

    print("\nInitializing Feedback Relevance System...")
    feedback_system = implement_feedback_relevance(pagerank_scores, search_engine.id_to_url, search_engine)

if __name__ == "__main__":
    main()

Starting crawling and indexing...
Crawling: https://www.bundesgesundheitsministerium.de/en/ (ID: 1)
Crawling: https://www.bundesgesundheitsministerium.de/en/index.html?tx_bmgsitepackage_bmg_article_list%5Baction%5D=list&tx_bmgsitepackage_bmg_article_list%5Bcontroller%5D=Article&tx_bmgsitepackage_bmg_article_list%5Bpage%5D=22&cHash=09012e25e8ea0ff0f3bfb90ec06d66db (ID: 2)
Crawling: https://www.bundesgesundheitsministerium.de/en/topics/digitalisation/digitalisation-strategy.html (ID: 3)
Crawling: https://www.bundesgesundheitsministerium.de/en/ministry/tasks-and-organisation/organisation-chart.html (ID: 4)
Crawling: https://www.bundesgesundheitsministerium.de/en/legal-notice.html (ID: 5)
Crawling: https://www.bundesgesundheitsministerium.de/en/ministry/leadership/pst-edgar-franke.html (ID: 6)
Crawling: https://www.bundesgesundheitsministerium.de/en/topics/digitalisation/digitalisation-strategy/the-transformation-approach-implementation-development-enhancement.html (ID: 7)
Crawling: https:

Output()


Displaying Feedback Interface for User 2...


Output()